In [25]:
import paho.mqtt.client as mqtt
#from azure.iot.device.aio import IoTHubDeviceClient
import random
import time

In [26]:
def generate_weather():
    t = random.uniform(-50.0,50.01)
    te = round(t,2)
    temp = str(te)+"°C"
    h = random.randint(0,100)
    hum = str(h) + "%"
    w_d = random.randint(0,360)
    wind_dir = str(w_d)+"°"
    w_i = random.randint(0,100)
    wind_int = str(w_i)+" m/s"
    r = random.randint(0,50)
    rain = str(r)+" mm/h"
    
    return temp, hum, wind_dir, wind_int, rain

'2020-03-20 17:36'

In [27]:
devices = ["foggia_1", "roma_1","rome_2"]

In [28]:
temp,hum,wind_dir,wind_int, rain = generate_weather()
print(temp)
print(hum)
print(wind_dir)
print(wind_int)
print(rain)

print(random.choice(clients))

0.7°C
64%
284°
45 m/s
47 mm/h
foggia_1


In [8]:
#from python to broker

def configuration():
    global hubAddress, deviceId, sharedAccessKey
    hubAddress = "CleverWeather.azure-devices.net"
    device_id = random.choice(clients)  #select a random element from client list
    if (device_id == "foggia_1"):
        sharedAccessKey = "mbjl7x2bqO3tJKHu+neU9MA7QY6ssLFtIs/EdK9h7SM="
    elif (device_id == "rome_1"):
        sharedAccessKey = "TxAmqOhF4ZHBTXQ3WBOzl+afiPZyC2PtesAvQYo1WHo="
    else:
        sharedAccessKey = "7ZelKlqvwoAty6uAvkVddJyTH5iOY1MQYpqjeI9JJE8="


def on_connect(client, userdata, flags, rc):
    print("Connected with result code: %s" % rc)

def on_disconnect(client, userdata, rc):
    print("Disconnected with result code: %s" % rc)

def on_publish(client, userdata, mid):
    print("Message {0} sent from {1}".format(str(mid), deviceId))

def publish():
    while True:
        try:
            temp,hum,wind_dir,wind_int, rain = generate_weather()
            hour = time.strftime("%Y-%m-%d %H:%M", time.gmtime())
            client.publish(topic = "topicmiometeo/temp", payload = temp,qos=1)
            client.publish(topic = "topicmiometeo/time", payload = hour,qos=1) 
            client.publish(topic = "topicmiometeo/hum", payload = hum,qos=1) 
            client.publish(topic = "topicmiometeo/wind_dir", payload = wind_dir,qos=1) 
            client.publish(topic = "topicmiometeo/wind_int", payload = wind_int,qos=1) 
            client.publish(topic = "topicmiometeo/rain", payload = rain,qos=1) 
            
            time.sleep(4)
        
        except KeyboardInterrupt:
            print("Sensors stopped")
            return

        except:
            print("Unexpected error")
            time.sleep(4)

configuration()
client = mqtt.Client(client_id="pyprog", clean_session=False, userdata=None, protocol="MQTTv311", transport="tcp")
#transport is "tcp" or “websockets” to send MQTT over WebSockets

client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_publish = on_publish
client.connect(hubAddress,port=8883, keepalive=60,bind_address="")
client.loop_start()

publish()

#client.username_pw_set()

#hostname to definee
"""client.publish(topic = clientid+"/temp", payload = temp,qos=1) 
client.publish(topic = clientid+"/hum", payload = hum,qos=1) 
client.publish(topic = clientid+"/wind_dir", payload = wind_dir,qos=1) 
client.publish(topic = clientid+"/wind_int", payload = wind_int,qos=1) 
client.publish(topic = clientid+"/rain", payload = rain,qos=1) """


ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione

In [48]:
def send_data(device: str):
    broker_url = "test.mosquitto.org"
    broker_port = 1883
    u = "domitix"
    p = "odioiot"

    client = mqtt.Client(client_id =device)

    def on_connect(client, userdata, flags, rc):
        #client connected
        if rc==0:
            print("connected OK Returned code=",rc)
        else:
            print("Bad connection Returned code=",rc)

    def on_disconnect(client, userdata, rc):
        #client disconnected
        print("Disconnected with result code: %s" % rc)

    def on_publish(client, userdata, mid):
        print("Message {0} sent from {1}".format(str(mid), deviceId))

    def on_message(client, userdata, message):
        print("Message Recieved")

    client.on_connect = on_connect
    client.on_disconnect = on_disconnect
    client.on_publish = on_publish
    client.on_message = on_message
    client.loop_start()
    client.connect(broker_url,broker_port)
    temp,hum,wind_dir,wind_int, rain = generate_weather()
    hour = time.strftime("%Y-%m-%d %H:%M", time.gmtime())
    client.publish(topic = "cleverweathertopic/"+device+"/temp", payload = temp,qos=1)
    client.publish(topic = "cleverweathertopic/"+device+"/time", payload = hour,qos=1) 
    client.publish(topic = "cleverweathertopic/"+device+"/hum", payload = hum,qos=1) 
    client.publish(topic = "cleverweathertopic/"+device+"/wind_dir", payload = wind_dir,qos=1) 
    client.publish(topic = "cleverweathertopic/"+device+"/wind_int", payload = wind_int,qos=1) 
    client.publish(topic = "cleverweathertopic/"+device+"/rain", payload = rain,qos=1) 
    client.loop_stop()
    #client.disconnect()
    #OPPURE UN UNICO MULTIPLE MESSAGE INVIATO OGNI TOT TEMPO

send_data("foggia_1")
send_data("roma_1")

'\nwhile True:\n    schedule.run_pending()\n    client.loop()\n    time.sleep(1)\n'